In [1]:
from cdag import Graph, load_data, read_table
import pandas as pd

In [2]:
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)

In [3]:
gr = pd.read_csv('ziscon_groepen.csv')
gr.loc[gr['CODE'] == 'ZH1294']

,CODE,SEARCHCODE,OMSCHR,MEMO,SOORT,BEDRIJF
2160,ZH1294,DBHHEMO,Dagbehandeling Verpleegkundige Hematologie (B&O),Cluster aangemaakt van de verpleegafdeling dag...,ZH00000017,NaN


In [4]:
#us = pd.read_csv('ziscon_user.csv',
#               dtype={'FUNKTIE':'string', 'BEHEERDER':'boolean', 'WACHTWOORD':'string', 'DISABLED': 'boolean',
#                     'UZINR': 'string', 'ROLE': 'string', 'WRONGCHECKS': 'string'})

In [5]:
tg = pd.read_csv('chipsoft_logic_table_guids.csv')
table_name = {}
for elt in tg.to_dict('records'):
    table_name[elt['classguid']] = elt['classid']

In [6]:
iv = pd.read_csv('config_instvars.csv')
ane = iv.loc[iv['OWNER'].str.contains('SPEANE')].copy()

In [7]:
ane.head()

,NAAM,OWNER,INSTTYPE,SPECCODE,VALUE,ETD_STATUS
4558,__CSSTD__CSSTD__,@GSPEANE,C,_CSSTDCSSTD_,"C""6,{272C4B94-7648-47BC-A739-435EC9BDBED5},F,C...",NaN
4559,AG_US_ANN,@GSPEANE,C,NaN,"C""0,{4C309B34-05C6-491E-929B-CEA51ED55FD7},F,C...",NaN
4560,AG_US_BEK,@GSPEANE,C,NaN,"C""0,{4C309B34-05C6-491E-929B-CEA51ED55FD7},F,""...",NaN
4561,AG_US_BERAFD,@GSPEANE,C,NaN,"C""0,{4C309B34-05C6-491E-929B-CEA51ED55FD7},F,C...",NaN
4562,AG_US_MAAK,@GSPEANE,C,NaN,"C""0,{4C309B34-05C6-491E-929B-CEA51ED55FD7},F,C...",NaN


In [8]:
occurring, missing = {}, set()
for elt in ane.to_dict('records'):
    try:
        guid = elt['VALUE'][0:-2].removeprefix('C"').replace('""', '"').split(',')[1]
    except IndexError:
        #print(elt['VALUE'])
        continue
    name = table_name.get(guid, '<unknown>')
    if name == '<unknown>':
        missing.add(name)
    else:
        occurring[guid] = name

#for key, value in occurring:
#    print(f"{key}: {value}")
#for key in missing:
#    print(key)

In [9]:
value = ane.to_dict('records')[0]['VALUE']
elts = value[0:-2].removeprefix('C"').replace('""', '"').split(',')
table_name[elts[1]]

'CSZISLIB_AANVAFD'

In [10]:
iv.groupby('INSTTYPE').count()

,NAAM,OWNER,SPECCODE,VALUE,ETD_STATUS
INSTTYPE,,,,,
C,5671,5671,2001,5671,0
D,351,351,235,351,0
G,6382,6383,4516,6379,0
L,70606,70606,47706,70606,0
U,904852,904852,269970,904852,0


In [11]:
# iv.loc[iv['INSTTYPE']=='D'].head(10)

In [12]:
wcs = wc.join(ws, how='inner')
#result = wcs.loc[~wcs['Disabled']].copy()
#result.to_csv('workcontext.csv')
wcs.loc[(wc['OwnerId']=='ZH0085') & wc['SettingId'].str.contains('AG_US_ANN')]

,OwnerId,OwnerType,SettingId,SettingSubId,SegmentClassId,ContextType,Inverted,SegmentId,Disabled
Id,,,,,,,,,
ZH00001509,ZH0085,G,AG_US_ANN,NaN,AGENDA_AGENDA,0,False,ZH0010,False
ZH00001509,ZH0085,G,AG_US_ANN,NaN,AGENDA_AGENDA,0,False,ZH0131,False
ZH00001509,ZH0085,G,AG_US_ANN,NaN,AGENDA_AGENDA,0,False,ZH0400,True
ZH00001509,ZH0085,G,AG_US_ANN,NaN,AGENDA_AGENDA,0,False,ZH0414,True
ZH00001509,ZH0085,G,AG_US_ANN,NaN,AGENDA_AGENDA,0,False,ZH0128,False


In [ ]:
iv = pd.read_csv('config_instvars.csv')
iv.loc[iv['NAAM'] == 'AG_US_ANN']